Refresh credentials and load the Weaviate IP. 

If the credentials expire, run this cell again to refresh them.

In [ ]:
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

%store -r WEAVIATE_IP

# Working with Weaviate

![Weaviate architecture](images/weaviate-explainer-architecture.png)

Connect to a Weaviate instance

In [ ]:
import weaviate
import os
from weaviate.classes.init import Auth

client = weaviate.connect_to_local(
    # STUDENT TODO:
    # Add WEAVIATE_IP as the first positional argument
    # ADD YOUR CODE HERE
    # These are the AWS credentials for Weaviate to use
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }
)

client.is_ready()

## Create collection with vectorizer

Create a collection for financial articles with separate embeddings for title and content.

[Docs - Collection configuration](https://weaviate.io/developers/weaviate/manage-data/collections)

![Weaviate collections](images/weaviate-explainer-collections.png)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

# Delete collection if it exists
if client.collections.exists("FinancialArticles"):
    client.collections.delete("FinancialArticles")

# Create collection with named vectors for title and content
client.collections.create(
    name="FinancialArticles",
    # Define property schema
    properties=[
        Property(name="article_title", data_type=DataType.TEXT),
        # STUDENT TODO:
        # Add two additional properties - "article" and "url"
        # Both should be of data_type DataType.TEXT
        # ADD YOUR CODE HERE
    ],
    # Configure separate embeddings for title and article content
    vector_config=[
        Configure.Vectors.text2vec_aws(
            name="title",
            source_properties=["article_title"],
            region="us-west-2",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"
        ),
        # STUDENT TODO:
        # Add a second vector configuration
        # using the same region, service, and model as above
        # Change the name to "content"
        # Change source properties to "article"
        # ADD YOUR CODE HERE
    ],
)

## Load financial articles data

Import the financial news dataset using batch processing for efficient loading.

In [ ]:
import pandas as pd
from tqdm import tqdm
from weaviate.util import generate_uuid5

# Load the financial articles dataset
df = pd.read_parquet("data/fin_news_articles_5000.parquet")

print(f"Loaded {len(df)} articles")
print("Sample data:")
df.head()

In [ ]:
# Get collection reference
# STUDENT TODO:
# Assign client.collections.use("FinancialArticles") to variable 'articles'
# ADD YOUR CODE HERE

In [ ]:
# Import data with batch processing
with articles.batch.fixed_size(batch_size=100) as batch:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        # Create object from dataframe row
        obj = {
            "article_title": row["article_title"],
            "article": row["article"],
            "url": row["url"] if "url" in row else ""
        }

        # Generate UUID to prevent duplicates
        uuid = generate_uuid5(row["article_title"] + str(row.get("url", "")))

        # STUDENT TODO:
        # Add the object to the batch
        # using batch.add_object
        # with properties=obj, uuid=uuid
        # ADD YOUR CODE HERE

        # Check for errors during import
        if batch.number_errors > 10:
            print("Too many errors during import")
            break

print(f"\nImported {len(articles)} articles")

## Check for import errors

In [ ]:
if len(articles.batch.failed_objects) > 0:
    print(f"Import completed with {len(articles.batch.failed_objects)} errors")
    for err in articles.batch.failed_objects[:5]:  # Show first 5 errors
        print(err)
else:
    print("Import completed successfully with no errors")

## Verify the data

Check that articles were imported correctly with embeddings.

In [ ]:
# Check collection size
# STUDENT TODO:
# Use the built-in len() function on the 'articles' collection to get the total number of articles
# ADD YOUR CODE HERE

In [ ]:
# View sample data
# STUDENT TODO:
# Fetch 3 articles
# Use articles.query.fetch_objects with `limit=3``
# START_SOLUTION
response = articles.query.fetch_objects(limit=3)
# END_SOLUTION

# Inspect the response
for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")

## Test basic search

Quick test to verify the collection works with semantic search.

In [ ]:
# Try semantic search on titles
# STUDENT TODO:
# Use `.query.near_text` to search for articles related to "technology earnings"
# Supply the target_vector argument as "title"
# Limit results to 3
# START_SOLUTION
response = articles.query.near_text(
    query="technology earnings",
    target_vector="title",
    limit=3
)
# END_SOLUTION


print("Search results for 'technology earnings':")
# Inspect the response
# STUDENT TODO:
# START_SOLUTION
for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")
# END_SOLUTION

## Close the client

Always close your connection when finished.

In [ ]:
# Close the client after use
client.close()